# Alternative (did not use)

In [28]:
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde, norm

rpt_data = pd.read_csv('qn2rpt.csv')
rpt_data_H = rpt_data['H_1'].dropna().values
rpt_data_I = rpt_data['I_2'].dropna().values
rpt_data_J = rpt_data['J_3'].dropna().values

def kde_mean_std(data):
    kde = gaussian_kde(data)
    x = np.linspace(min(data), max(data), 1000)
    dx = x[1] - x[0]
    mean = np.sum(x * kde(x)) * dx
    std = np.sqrt(np.sum((x - mean)**2 * kde(x)) * dx)
    return mean, std

mean_H, std_H = np.mean(rpt_data_H), np.std(rpt_data_H)
mean_I, std_I = kde_mean_std(rpt_data_I)  
mean_J, std_J = kde_mean_std(rpt_data_J) 
print(mean_H, std_H)
print(mean_I, std_I)
print(mean_J, std_J)

utilization = {'H': 0.85, 'I': 0.75, 'J': 0.60}
tools_df = pd.read_csv('tools.csv', index_col=0)
tools_df.index = tools_df.index.str.replace('’', '_')

loading_data = {
    'Q1_26': {'H': 12000, 'I': 5000, 'J': 1000},
    'Q2_26': {'H': 12006, 'I': 6900, 'J': 3500},
    'Q3_26': {'H': 9640, 'I': 8969, 'J': 6000},
    'Q4_26': {'H': 8178, 'I': 9486, 'J': 7096},
    'Q1_27': {'H': 6981, 'I': 10080, 'J': 7994},
    'Q2_27': {'H': 6981, 'I': 10080, 'J': 7994},
    'Q3_27': {'H': 6878, 'I': 9917, 'J': 8033},
    'Q4_27': {'H': 6860, 'I': 10078, 'J': 7995},
}

quarterly_z_scores = {workstation: {} for workstation in ['H', 'I', 'J']}
quarterly_failure_probabilities = {workstation: {} for workstation in ['H', 'I', 'J']}

for quarter, load in loading_data.items():
    tools = {
        'H': tools_df.loc[quarter, 'H'],
        'I': tools_df.loc[quarter, 'I'],
        'J': tools_df.loc[quarter, 'J']
    }
    print(tools)
    
    for node in ['H', 'I', 'J']:
        available_time = 7 * 24 * 60 * utilization[node] * tools[node]
        
        mean = mean_H if node == 'H' else (mean_I if node == 'I' else mean_J)
        std = std_H if node == 'H' else (std_I if node == 'I' else std_J)
        total_time = load[node] * mean
        std_total = std * np.sqrt(load[node])  
        
        z_score = (total_time - available_time) / std_total
        quarterly_z_scores[node][quarter] = z_score
        
        failure_probability = 1 - norm.cdf(z_score)
        quarterly_failure_probabilities[node][quarter] = round(failure_probability, 5)

print("Z-scores and Failure Probabilities for the first week of each quarter:")
for node in ['H', 'I', 'J']:
    print(f"\nWorkstation {node}:")
    for quarter, z in quarterly_z_scores[node].items():
        prob = quarterly_failure_probabilities[node][quarter]
        failrate = 1-prob
        quarterly_failrate = 1 - (1 - failrate) ** 13
        print(f"{quarter}: Z = {z:.2f}, failrate: {failrate:.5f}, failrate/Q: {quarterly_failrate:.5f}")

2.1316261381378303 0.5004601541022468
5.9305013150648795 3.2177313130728105
2.7590386740627886 1.8954199731477088
{'H': 3, 'I': 4, 'J': 1}
{'H': 3, 'I': 6, 'J': 2}
{'H': 3, 'I': 8, 'J': 3}
{'H': 3, 'I': 8, 'J': 3}
{'H': 3, 'I': 8, 'J': 3}
{'H': 3, 'I': 8, 'J': 3}
{'H': 3, 'I': 8, 'J': 3}
{'H': 3, 'I': 8, 'J': 3}
Z-scores and Failure Probabilities for the first week of each quarter:

Workstation H:
Q1_26: Z = -2.27, failrate: 0.01158, failrate/Q: 0.14051
Q2_26: Z = -2.04, failrate: 0.02083, failrate/Q: 0.23940
Q3_26: Z = -104.91, failrate: 0.00000, failrate/Q: 0.00000
Q4_26: Z = -182.77, failrate: 0.00000, failrate/Q: 0.00000
Q1_27: Z = -258.84, failrate: 0.00000, failrate/Q: 0.00000
Q2_27: Z = -258.84, failrate: 0.00000, failrate/Q: 0.00000
Q3_27: Z = -266.06, failrate: 0.00000, failrate/Q: 0.00000
Q4_27: Z = -267.33, failrate: 0.00000, failrate/Q: 0.00000

Workstation I:
Q1_26: Z = -2.58, failrate: 0.00491, failrate/Q: 0.06198
Q2_26: Z = -16.61, failrate: 0.00000, failrate/Q: 0.00000


# We decided to use this code


In [29]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.mixture import GaussianMixture

rpt_data = pd.read_csv('qn2rpt.csv')
rpt_data_H = rpt_data['H_1'].dropna().values
rpt_data_I = rpt_data['I_2'].dropna().values
rpt_data_J = rpt_data['J_3'].dropna().values

def monte_carlo_mean_std(data, num_samples=10000):
    gmm = GaussianMixture(n_components=2, random_state=0).fit(data.reshape(-1, 1))
    
    samples, _ = gmm.sample(num_samples)
    
    mean = np.mean(samples)
    std = np.std(samples)
    
    return mean, std

mean_H, std_H = np.mean(rpt_data_H), np.std(rpt_data_H)  
mean_I, std_I = monte_carlo_mean_std(rpt_data_I)  
mean_J, std_J = monte_carlo_mean_std(rpt_data_J)  
print(mean_H, std_H)
print(mean_I, std_I)
print(mean_J, std_J)

utilization = {'H': 0.85, 'I': 0.75, 'J': 0.60}

tools_df = pd.read_csv('tools.csv', index_col=0)
tools_df.index = tools_df.index.str.replace('’', '_')

loading_data = {
    'Q1_26': {'H': 12000, 'I': 5000, 'J': 1000},
    'Q2_26': {'H': 12006, 'I': 6900, 'J': 3500},
    'Q3_26': {'H': 9640, 'I': 8969, 'J': 6000},
    'Q4_26': {'H': 8178, 'I': 9486, 'J': 7096},
    'Q1_27': {'H': 6981, 'I': 10080, 'J': 7994},
    'Q2_27': {'H': 6981, 'I': 10080, 'J': 7994},
    'Q3_27': {'H': 6878, 'I': 9917, 'J': 8033},
    'Q4_27': {'H': 6860, 'I': 10078, 'J': 7995},
}

quarterly_z_scores = {workstation: {} for workstation in ['H', 'I', 'J']}
quarterly_failure_probabilities = {workstation: {} for workstation in ['H', 'I', 'J']}

for quarter, load in loading_data.items():
    tools = {
        'H': tools_df.loc[quarter, 'H'],
        'I': tools_df.loc[quarter, 'I'],
        'J': tools_df.loc[quarter, 'J']
    }
    print(tools)
    
    for node in ['H', 'I', 'J']:
        available_time = 7 * 24 * 60 * utilization[node] * tools[node]
        
        mean = mean_H if node == 'H' else (mean_I if node == 'I' else mean_J)
        std = std_H if node == 'H' else (std_I if node == 'I' else std_J)
        total_time = load[node] * mean
        std_total = std * np.sqrt(load[node])  
        
        z_score = (total_time - available_time) / std_total
        quarterly_z_scores[node][quarter] = z_score
        
        failure_probability = 1 - norm.cdf(z_score)
        quarterly_failure_probabilities[node][quarter] = round(failure_probability, 5)

print("\nZ-scores and Failure Probabilities for the first week of each quarter:")
for node in ['H', 'I', 'J']:
    print(f"\nWorkstation {node}:")
    for quarter, z in quarterly_z_scores[node].items():
        prob = quarterly_failure_probabilities[node][quarter]
        failrate = 1-prob
        quarterly_failrate = 1 - (1 - failrate) ** 13
        print(f"{quarter}: Z = {z:.2f}, failrate: {failrate:.5f}, failrate/Q: {quarterly_failrate:.5f}")

2.1316261381378303 0.5004601541022468
5.9541527208528295 3.2751666482272364
3.0172045594685777 2.053075623272222
{'H': 3, 'I': 4, 'J': 1}
{'H': 3, 'I': 6, 'J': 2}
{'H': 3, 'I': 8, 'J': 3}
{'H': 3, 'I': 8, 'J': 3}
{'H': 3, 'I': 8, 'J': 3}
{'H': 3, 'I': 8, 'J': 3}
{'H': 3, 'I': 8, 'J': 3}
{'H': 3, 'I': 8, 'J': 3}

Z-scores and Failure Probabilities for the first week of each quarter:

Workstation H:
Q1_26: Z = -2.27, failrate: 0.01158, failrate/Q: 0.14051
Q2_26: Z = -2.04, failrate: 0.02083, failrate/Q: 0.23940
Q3_26: Z = -104.91, failrate: 0.00000, failrate/Q: 0.00000
Q4_26: Z = -182.77, failrate: 0.00000, failrate/Q: 0.00000
Q1_27: Z = -258.84, failrate: 0.00000, failrate/Q: 0.00000
Q2_27: Z = -258.84, failrate: 0.00000, failrate/Q: 0.00000
Q3_27: Z = -266.06, failrate: 0.00000, failrate/Q: 0.00000
Q4_27: Z = -267.33, failrate: 0.00000, failrate/Q: 0.00000

Workstation I:
Q1_26: Z = -2.03, failrate: 0.02137, failrate/Q: 0.24484
Q2_26: Z = -15.72, failrate: 0.00000, failrate/Q: 0.00000


# Alternative (did not use)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.mixture import GaussianMixture

rpt_data = pd.read_csv('qn2rpt.csv')
rpt_data_H = rpt_data['H_1'].dropna().values
rpt_data_I = rpt_data['I_2'].dropna().values
rpt_data_J = rpt_data['J_3'].dropna().values

mean_H = np.mean(rpt_data_H)
std_H  = np.std(rpt_data_H)

gmm_I = GaussianMixture(n_components=2, random_state=0).fit(rpt_data_I.reshape(-1, 1))
gmm_J = GaussianMixture(n_components=2, random_state=0).fit(rpt_data_J.reshape(-1, 1))

def simulate_total_time(data, load, num_simulations=5000, use_gmm=False, gmm_model=None):
    """
    Simulate the total processing time for a given workload (number of jobs applied to a workstation).
    
    Parameters:
      data            : historical processing time data (np.array)
      load            : integer count of jobs to process in the quarter (or week)
      num_simulations : number of Monte Carlo simulation trials
      use_gmm         : if True, then use the provided gmm_model to draw samples.
      gmm_model       : a fitted GaussianMixture model (if use_gmm is True)
    
    Returns:
      total_times   : an array with the total processing time from each simulation trial.
    """
    if (not use_gmm):
        samples = np.random.normal(loc=np.mean(data), scale=np.std(data), 
                                   size=(num_simulations, load))
        total_times = samples.sum(axis=1)
    else:
        total_times = np.zeros(num_simulations)
        for i in range(num_simulations):
            samples, _ = gmm_model.sample(load)
            total_times[i] = samples.sum()
    return total_times

utilization = {'H': 0.85, 'I': 0.75, 'J': 0.60}

tools_df = pd.read_csv('tools.csv', index_col=0)
tools_df.index = tools_df.index.str.replace('’', '_')

loading_data = {
    'Q1_26': {'H': 12000, 'I': 5000, 'J': 1000},
    'Q2_26': {'H': 12006, 'I': 6900, 'J': 3500},
    'Q3_26': {'H': 9640,  'I': 8969, 'J': 6000},
    'Q4_26': {'H': 8178,  'I': 9486, 'J': 7096},
    'Q1_27': {'H': 6981,  'I': 10080,'J': 7994},
    'Q2_27': {'H': 6981,  'I': 10080,'J': 7994},
    'Q3_27': {'H': 6878,  'I': 9917, 'J': 8033},
    'Q4_27': {'H': 6860,  'I': 10078,'J': 7995},
}

quarterly_failure_probabilities_sim = {ws: {} for ws in ['H', 'I', 'J']}

NUM_SIMULATIONS = 100000

for quarter, load in loading_data.items():
    
    tools = {
        'H': tools_df.loc[quarter, 'H'],
        'I': tools_df.loc[quarter, 'I'],
        'J': tools_df.loc[quarter, 'J']
    }
    print(f"\nQuarter: {quarter}, Tools: {tools}")
    
    for node in ['H', 'I', 'J']:
        available_time = 7 * 24 * 60 * utilization[node] * tools[node]
        
        node_load = load[node]
        
        if node == 'H':
            total_times = simulate_total_time(rpt_data_H, node_load,
                                              num_simulations=NUM_SIMULATIONS,
                                              use_gmm=False)
        elif node == 'I':
            total_times = simulate_total_time(rpt_data_I, node_load,
                                              num_simulations=NUM_SIMULATIONS,
                                              use_gmm=True, gmm_model=gmm_I)
        else:  
            total_times = simulate_total_time(rpt_data_J, node_load,
                                              num_simulations=NUM_SIMULATIONS,
                                              use_gmm=True, gmm_model=gmm_J)

        failure_prob = (total_times > available_time).mean()
        quarterly_failure_probabilities_sim[node][quarter] = round(failure_prob, 5)
        
        sim_mean = total_times.mean()
        sim_std = total_times.std()
        print(f" Workstation {node}: Load = {node_load}, Available = {available_time:.0f} minutes")
        print(f"   Simulated total time: mean = {sim_mean:.1f}, std = {sim_std:.1f}")
        print(f"   Weekly failure probability (total_time > available): {failure_prob:.5f}")

print("\nSimulation-based Failure Probabilities (and compounded quarterly failrate):")
for node in ['H', 'I', 'J']:
    print(f"\nWorkstation {node}:")
    for quarter, weekly_fail in quarterly_failure_probabilities_sim[node].items():
        quarterly_failrate = 1 - (1 - weekly_fail) ** 13
        print(f" {quarter}: weekly fail = {weekly_fail:.5f}, compounded quarterly fail = {quarterly_failrate:.5f}")


Quarter: Q1_26, Tools: {'H': 3, 'I': 4, 'J': 1}
 Workstation H: Load = 12000, Available = 25704 minutes
   Simulated total time: mean = 25579.5, std = 54.6
   Weekly failure probability (total_time > available): 0.01153
 Workstation I: Load = 5000, Available = 30240 minutes
   Simulated total time: mean = 29797.5, std = 0.0
   Weekly failure probability (total_time > available): 0.00000
 Workstation J: Load = 1000, Available = 6048 minutes
   Simulated total time: mean = 3011.5, std = 0.0
   Weekly failure probability (total_time > available): 0.00000

Quarter: Q2_26, Tools: {'H': 3, 'I': 6, 'J': 2}
 Workstation H: Load = 12006, Available = 25704 minutes
   Simulated total time: mean = 25592.4, std = 54.8
   Weekly failure probability (total_time > available): 0.02045
 Workstation I: Load = 6900, Available = 45360 minutes
   Simulated total time: mean = 41179.3, std = 0.0
   Weekly failure probability (total_time > available): 0.00000
 Workstation J: Load = 3500, Available = 12096 min